In [1]:

from typing import Sequence
import os
import numpy as np

from scipy.ndimage import label
from scipy import ndimage
import cv2
import torch
import torchvision
import torchvision.transforms as transforms
from motpy import Detection, ModelPreset, MultiObjectTracker, NpImage
from motpy.core import setup_logger
from motpy.detector import BaseObjectDetector
from motpy.testing_viz import draw_detection, draw_track
from motpy.utils import ensure_packages_installed




In [2]:
path = os.path.abspath("video/san_pham1.mp4")
i =0

In [3]:
def read_video_file(video_path: str):
    video_path = os.path.expanduser(video_path)
    cap = cv2.VideoCapture(video_path)
    video_fps = float(cap.get(cv2.CAP_PROP_FPS))
    return cap, video_fps

In [4]:
def erosion(src):
    kernel = np.ones((5, 5), np.uint8)
    erosion_dst = cv2.erode(src, kernel)
    return erosion_dst

In [5]:

def dilatation(src):
    kernel = np.ones((5, 5), np.uint8)
    dilatation_dst = cv2.dilate(src, kernel)
    return dilatation_dst


In [6]:
def label_array(array_binary):
    a = np.array(array_binary)
    labeled_array, num_features = label(a)
    return labeled_array, num_features

In [7]:
def finding_object(array_object):
    return ndimage.find_objects(array_object)


In [8]:
def processImage(slices,i):

    out_detections = []
    if slices is not None:
        for label_slice in slices:
            bboxx = label_slice[1]
            bboxy = label_slice[0]
            ymin, ymax, xmin, xmax = bboxy.start, bboxy.stop, bboxx.start, bboxx.stop
            i=i+1
            out_detections.append(Detection(box=[xmin, ymin, xmax, ymax], score=1,class_id = i))
        print(i)   
        return out_detections

In [9]:
def run(video_path, video_downscale: float = 0.4, viz_wait_ms: int = 1,i=0):
    model_spec = {'order_pos': 1, 'dim_pos': 2,
                  'order_size': 0, 'dim_size': 2,
                  'q_var_pos': 5000., 'r_var_pos': 0.1}


    cap, cap_fps = read_video_file(video_path)

    dt = 1 / cap_fps # assume 15 fps
    tracker = MultiObjectTracker(dt=dt, model_spec=model_spec)
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, fx=video_downscale, fy=video_downscale, dsize=None, interpolation=cv2.INTER_AREA)
        # chuyen anh sang anh xam
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blob = cv2.dnn.blobFromImage(gray, 1.0, (300, 300), [104, 117, 123], False, False)
        # chuyen anh sang nhi phan
        ret, threshold1 = cv2.threshold(gray, 210, 255, cv2.THRESH_BINARY)
        threshold_dilation = dilatation(threshold1)
        threshold_erosion = erosion(threshold_dilation)

        labeled_array, num_features = label_array(threshold_erosion)
        slices=finding_object(labeled_array)
        detections = processImage(slices,i)
        # print('detections process: ', detections)

        _ = tracker.step(detections)
        tracks = tracker.active_tracks(min_steps_alive=45)

        for det in detections:
            draw_detection(frame, det)


        for track in tracks:
            draw_track(frame, track)


        cv2.imshow('frame', frame)
        c = cv2.waitKey(viz_wait_ms)
        if c == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [10]:
run(video_path=path,i=0)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
3
3
4
4
4
1
3
2
2
1
4
3
3
3
2
2
2
2
3
3
4
1
3
3
4
4
4
3
4
5
4
5
5
4
4
5
5
7
6
5
4
5
8
7
6
5
7
7
5
7
8
7
7
7
7
7
6
8
7
9
6
7
8
7
9
9
7
8
8
9
8
9
9
9
10
9
7
10
8
7
6
7
7
8
7
10
9
10
6
8
9
14
12
11
10
9
10
10
10
13
9
14
11
11
12
10
14
13
19
14
13
11
12
12
9
7
14
13
13
12
12
12
12
14
15
12
13
12
9
9
9
9
6
9
8
6
5
7
6
8
9
7
8
9
8
9
11
11
7
7
11
8
12
11
11
9
13
11
7
11
7
7
8
8
7
8
8
7
10
10
8
8
8
7
9
8
10
12
9
7
8
7
7
8
6
8
9
9
8
8
8
8
9
7
8
9
7
10
10
8
9
7
8
8
8
5
6
8
8
8
8
8
9
9
8
7
7
8
7
8
12
10
11
8
8
12
11
9
9
9
6
6
7
5
8
6
7
6
5
4
6
7
7
5
5
4
5
6
6
4
4
5
6
5
5
5
7
8
7
4
7
6
7
5
5
6
6
7
8
9
5
7
5
3
3
4
3
4
3
3
4
3
2
2
2
3
3
2
2
2
2
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
